In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import myutils

OS:  Linux-3.10.0-1127.8.2.el7.x86_64-x86_64-with-glibc2.10
python:  3.8.3 | packaged by conda-forge | (default, Jun  1 2020, 17:43:00) 
[GCC 7.5.0]
sklearn:  0.24.1
numpy:  1.19.2
pandas:  1.2.3
matplotlib:  3.3.4
tensorflow:  2.4.1
Sat May  7 13:10:40 2022


In [1]:
!pwd

/home/mlpan/Courses/Capstone-Birds-Action-Recognition


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.test.is_built_with_cuda()

True

In [4]:
myutils.GPU()

GPU:
 Sat May  7 13:10:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:0B:00.0 Off |                    0 |
| N/A   31C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-

In [20]:
myutils.CPU()

CPU:
 processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU E5-2680 v4 @ 2.40GHz
stepping	: 1
microcode	: 0xb000020
cpu MHz		: 1200.000
cache size	: 35840 KB
physical id	: 0
siblings	: 14
core id		: 0
cpu cores	: 14
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 20
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc aperfmperf eagerfpu pni pclmulqdq dtes64 monitor ds_cpl vmx smx est tm2 ssse3 sdbg fma cx16 xtpr pdcm pcid dca sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand lahf_lm abm 3dnowprefetch epb cat_l3 cdp_l3 invpcid_single intel_ppin intel_pt tpr_shadow vnmi flexpriority ept vpid fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm cqm rdt_a rdseed adx xsaveopt cqm_llc cqm_occup_llc 

# Benchmark

In [11]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
# scaling image values between 0-1
X_train_scaled = X_train/255
X_test_scaled = X_test/255
# one hot encoding labels
y_train_encoded = tf.keras.utils.to_categorical(y_train, num_classes = 10, dtype = 'float32')
y_test_encoded = tf.keras.utils.to_categorical(y_test, num_classes = 10, dtype = 'float32')

In [12]:
X_train.shape, X_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [17]:
def get_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(32,32,3)),
        tf.keras.layers.Dense(3000, activation='relu'),
        tf.keras.layers.Dense(1000, activation='relu'),
        tf.keras.layers.Dense(10, activation='sigmoid')    
    ])
    model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [18]:
%%timeit -n1 -r1
# CPU
with tf.device('/CPU:0'):
    model_cpu = get_model()
    model_cpu.fit(X_train_scaled, y_train_encoded, epochs = 10)

Epoch 1/10
1563/1563 [==============================] - 32s 20ms/step - loss: 1.9353 - accuracy: 0.3050
Epoch 2/10
1563/1563 [==============================] - 31s 20ms/step - loss: 1.6421 - accuracy: 0.4214
Epoch 3/10
1563/1563 [==============================] - 31s 20ms/step - loss: 1.5552 - accuracy: 0.4510
Epoch 4/10
1563/1563 [==============================] - 31s 20ms/step - loss: 1.4804 - accuracy: 0.4787
Epoch 5/10
1563/1563 [==============================] - 31s 20ms/step - loss: 1.4245 - accuracy: 0.5001
Epoch 6/10
1563/1563 [==============================] - 31s 20ms/step - loss: 1.3879 - accuracy: 0.5162
Epoch 7/10
1563/1563 [==============================] - 31s 20ms/step - loss: 1.3393 - accuracy: 0.5279
Epoch 8/10
1563/1563 [==============================] - 31s 20ms/step - loss: 1.3095 - accuracy: 0.5395
Epoch 9/10
1563/1563 [==============================] - 31s 20ms/step - loss: 1.2789 - accuracy: 0.5534
Epoch 10/10
1563/1563 [==============================] - 31s 20m

In [19]:
%%timeit -n1 -r1
# GPU
with tf.device('/GPU:0'):
    model_gpu = get_model()
    model_gpu.fit(X_train_scaled, y_train_encoded, epochs = 10)

Epoch 1/10
1563/1563 [==============================] - 7s 3ms/step - loss: 1.9257 - accuracy: 0.3053
Epoch 2/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.6440 - accuracy: 0.4189
Epoch 3/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.5534 - accuracy: 0.4513
Epoch 4/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.4897 - accuracy: 0.4741
Epoch 5/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.4350 - accuracy: 0.4941
Epoch 6/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.3854 - accuracy: 0.5103
Epoch 7/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.3441 - accuracy: 0.5290
Epoch 8/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.3098 - accuracy: 0.5425
Epoch 9/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.2813 - accuracy: 0.5501
Epoch 10/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.250